<a href="https://colab.research.google.com/github/Aggraj/Deep-Learning-CS-6910/blob/main/FFNNv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np

class Feedforwardneuralnetwork:

    def __init__(self,n_inputs,n_hidden,n_outputs):

        self.n_inputs   = n_inputs
        self.n_outputs  = n_outputs
        self.n_hidden   = n_hidden
        self.weights    = []
        self.biases     = []

        layers = [self.n_inputs] + self.n_hidden + [self.n_outputs]
        for i in range(len(n_hidden)+1):
            self.weights.append(np.random.randn(layers[i+1],layers[i]))
            self.biases.append(np.random.randn(layers[i+1],1))



    def sigmoid(self,x):
        return 1 / ( 1 + np.exp(-x))

    def softmax(self,x):
        soft = np.zeros(x.shape)
        for i in range(0, x.shape[1]):
            numr = np.exp(x[:, i])
            soft[:, i] = numr/np.sum(numr)
        return soft

    def forward_propagation(self,input):

        self.intermidiate_inputs = []
        self.post_outputs  = []
        W      = self.weights
        b      = self.biases

        k=0
        self.intermidiate_inputs.append(np.matmul(W[k],input)+b[k])
        self.post_outputs.append(self.sigmoid(self.intermidiate_inputs[k]))

        for k in range(1,len(self.n_hidden)):
            self.intermidiate_inputs.append(np.matmul(W[k],self.post_outputs[k-1])+b[k])
            self.post_outputs.append(self.sigmoid(self.intermidiate_inputs[k]))

        k=len(self.n_hidden)
        self.intermidiate_inputs.append(np.matmul(W[k],self.post_outputs[k-1])+b[k])
        self.post_outputs.append(self.softmax(self.intermidiate_inputs[k]))

        return self.post_outputs[-1]

    def back_propagation(self,train_images,train_labels):

        g_weights = [0]*(len(self.weights))
        g_biases  = [0]*(len(self.biases))
        g_a       = [0]*(len(self.n_hidden)+1)
        g_h       = [0]*(len(self.n_hidden)+1)
        n_samples = train_images.shape[0]  # Change depending on the dimensions of data


        for k in reversed(range(len(self.n_hidden)+1)):
            if k == len(self.n_hidden):
                g_a[k] = self.post_outputs[k] - train_labels  # keep or remove T depending on the dimensions of data
                #g_weights[k] = (1/n_samples)*np.matmul(g_a[k],self.post_outputs[k-1].T)
                #g_biases[k]  = (1/n_samples)*np.sum(g_a[k], axis=1, keepdims = True)
            else:
                g_h[k] = (1/n_samples)*np.matmul(self.weights[k+1].T,g_a[k+1])
                g_a[k] = (1/n_samples)*np.multiply(g_h[k],np.multiply(self.sigmoid(self.intermidiate_inputs[k]),(1-self.sigmoid(self.intermidiate_inputs[k]))))

            if k == 0:
                g_weights[k] = (1/n_samples)*np.matmul(g_a[k],train_images.T)
            else:
                g_weights[k] = (1/n_samples)*np.matmul(g_a[k],self.post_outputs[k-1].T)

            g_biases[k]  = (1/n_samples)*np.sum(g_a[k], axis=1, keepdims = True)
        return g_weights,g_biases



    def train_model(self,train_images,train_labels,epochs,learning_rate,opt='gd',gamma = 0.9): 
         
        pre_delta_w = np.multiply(self.weights,0)
        pre_delta_b = np.multiply(self.biases,0)
        for i in range(epochs+1):
            output =  self.forward_propagation(train_images)
            g_weights,g_biases = self.back_propagation(train_images,train_labels)
            if opt == 'gd':
                delta_w = np.multiply(learning_rate,g_weights)
                delta_b = np.multiply(learning_rate,g_biases)
          
            if opt == 'mgd':
                 delta_w = np.multiply(gamma,pre_delta_w) + np.multiply(learning_rate,g_weights)
                 delta_b = np.multiply(gamma,pre_delta_b) + np.multiply(learning_rate,g_biases)
                 pre_delta_w = delta_w
                 pre_delta_b = delta_b
            
            self.weights = self.weights - delta_w
            self.biases  = self.biases  - delta_b
            train_loss = -np.sum(np.multiply(train_labels,np.log(output)))/train_labels.shape[1]    
            print('training_loss for epoch {} = {}'.format(i,train_loss))

        return train_loss

In [ ]:
import tensorflow as tf
from sklearn.metrics import log_loss
from keras.datasets import fashion_mnist
output_classes = 10
Model1  = Feedforwardneuralnetwork(28*28,[256,128],output_classes)
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
n_samples = train_images.shape[0]
train_images = train_images.reshape(n_samples,-1)
train_imag = train_images[:50000,:]
train_val_images = train_images[50000:,:]
labels = np.zeros((train_labels.shape[0],output_classes))
for i in range(train_labels.shape[0]):
  e = [0.0]*output_classes
  e[train_labels[i]] = 1.0
  labels[i] = e
train_label = labels[:50000,:]
train_val_labels = labels[50000:,:]
mean = train_imag.mean(axis=0)
std  = train_imag.std(axis = 0)
train_imag = (train_imag - mean)/std
train_val_images = (train_val_images - mean)/std
epochs = 100
learning_rate = 0.01
(tr_loss) = Model1.train_model(train_imag.T,train_label.T,epochs,learning_rate,'mgd')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local

training_loss for epoch 0 = 12.52498160057375
training_loss for epoch 1 = 9.396840609910635
training_loss for epoch 2 = 8.884828446841588
training_loss for epoch 3 = 9.667867905682105
training_loss for epoch 4 = 11.043979416117114
training_loss for epoch 5 = 8.515555168245886
training_loss for epoch 6 = 8.10901327723656
training_loss for epoch 7 = 7.9158423416613495
training_loss for epoch 8 = 6.48474668426841
training_loss for epoch 9 = 4.276613885333148
training_loss for epoch 10 = 3.228022665032938
training_loss for epoch 11 = 3.552669023455071
training_loss for epoch 12 = 4.247158732233213
training_loss for epoch 13 = 3.6566222335902387
training_loss for epoch 14 = 2.53488601123684
training_loss for epoch 15 = 2.9095678589369096
training_loss for epoch 16 = 2.9007100517152122
training_loss for epoch 17 = 2.680317512449394
training_loss for epoch 18 = 2.491894835082479


In [15]:

output = Model1.forward_propagation(train_imag.T)
out_class=(np.argmax(output,axis=0))
target_class=(np.argmax(train_label,axis=1))
acc2 = 100*np.sum(out_class==target_class)/output.shape[1]
print(acc2)

45.212


In [ ]:
train_label.shape

(50000, 10)

In [ ]:
n_inputs = 3
n_hidden = [4,5,100]
n_outputs = 6
nn = Feedforwardneuralnetwork(n_inputs,n_hidden,n_outputs)
input = np.array([[1,2,3],[2,6,2]]).T
output= np.array([[1,5,3,1,3,3],[1,3,3,4,2,3]]).T
A = nn.forward_propagation(input)
B = nn.back_propagation(input,output)
(tr_loss) = nn.train_model(input,output,epochs,learning_rate)

training_loss for epoch 0 = 73.98759052408417
training_loss for epoch 1 = 70.25713510358364
training_loss for epoch 2 = 66.9700185524711
training_loss for epoch 3 = 64.21729677564682
training_loss for epoch 4 = 62.07692073639605
training_loss for epoch 5 = 60.59423342999598
training_loss for epoch 6 = 59.7657188865998
training_loss for epoch 7 = 59.53497053881626
training_loss for epoch 8 = 59.80534129500605
training_loss for epoch 9 = 60.46340977911884
training_loss for epoch 10 = 61.40128947655657
training_loss for epoch 11 = 62.52967817473877
training_loss for epoch 12 = 63.781293785854615
training_loss for epoch 13 = 65.10852900321177
training_loss for epoch 14 = 66.47907374302748
training_loss for epoch 15 = 67.8716248504022
training_loss for epoch 16 = 69.27245537249969
training_loss for epoch 17 = 70.67292247119018
training_loss for epoch 18 = 72.06774561994091
training_loss for epoch 19 = 73.45384794060777
training_loss for epoch 20 = 74.82958672472844
training_loss for epoch 2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
